In [3]:
import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go

# Hent data fra SSB API
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {
    "query": [],
    "response": {
        "format": "json-stat2"
    }
}
response = requests.post(api_url, json=query)

if response.status_code == 200:
    data = response.json()

    # Trekk ut dimensjoner og verdier
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values

    # Pivotér
    df_pivot = df.pivot_table(
        index="Tid",
        columns=["VareGrupper2", "ContentsCode"],
        values="Value",
        aggfunc="sum"
    ).reset_index()

    # Gi forståelige kolonnenavn
    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

    # Hent ut år fra Tid (første 4 tegn)
    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)

    # Aggregér data per år (gjennomsnitt)
    df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)

    # Plotly-graf
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_agg["År"],
        y=df_agg["Fersk laks - Vekt (tonn)"],
        name="Fersk laks - Vekt (tonn)",
        yaxis="y1",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=df_agg["År"],
        y=df_agg["Fersk laks - Kilospris"],
        name="Fersk laks - Kilospris (kr/kg)",
        yaxis="y2",
        mode="lines+markers"
    ))

    fig.update_layout(
        title="Fersk laks – Årlig gjennomsnitt av vekt og kilospris",
        xaxis=dict(title="År"),
        yaxis=dict(title="Vekt (tonn)", side="left"),
        yaxis2=dict(title="Kilospris (kr/kg)", overlaying="y", side="right"),
        legend=dict(x=0.01, y=0.99),
        hovermode="x unified",
        template="plotly_white",
        height=600
    )

    fig.show()

else:
    print(f"Feil ved henting av data: {response.status_code}")
    print(response.text)


In [11]:
import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go

# --- LES CSV-FILEN ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)

csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000


# --- HENT OG BEHANDLE SSB-DATA ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)

if response.status_code == 200:
    data = response.json()
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values

    df_pivot = df.pivot_table(
        index="Tid",
        columns=["VareGrupper2", "ContentsCode"],
        values="Value",
        aggfunc="sum"
    ).reset_index()

    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
    df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)

    # --- FLETT DATAENE ---
    df_agg = df_agg[df_agg["År"] >= 2015]  # Behold kun data fra og med 2015

    merged = pd.merge(df_agg, csv_data, on="År", how="left")

    # --- INTERAKTIV GRAF ---
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged["År"],
        y=merged["Fersk laks - Vekt (tonn)"],
        name="Fersk laks - Vekt (tonn)",
        yaxis="y1",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=merged["År"],
        y=merged["Fersk laks - Kilospris"],
        name="Fersk laks - Kilospris (kr/kg)",
        yaxis="y2",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=merged["År"],
        y=merged["Eksportverdi (mrd NOK)"],
        name="Eksportverdi hittil i år (mrd NOK)",
        yaxis="y3",
        mode="lines+markers"
    ))

    fig.update_layout(
        title="Fersk laks – Vekt, Kilospris og Eksportverdi (per år)",
        xaxis=dict(title="År"),
        yaxis=dict(title="Vekt (tonn)", side="left", position=0.0),
        yaxis2=dict(title="Kilospris (kr/kg)", overlaying="y", side="right", position=1.0),
        yaxis3=dict(title="Eksportverdi (mrd NOK)", overlaying="y", anchor="free", side="right", position=0.85),
        legend=dict(x=0.01, y=0.99),
        hovermode="x unified",
        template="plotly_white",
        height=650
    )

    fig.show()


#predikær analyse 
from sklearn.linear_model import LinearRegression
import numpy as np


# --- Filtrer bort rader med NaN i eksportverdi ---
filtered = merged.dropna(subset=["Eksportverdi (mrd NOK)"])
X = filtered[["År"]]
y = filtered["Eksportverdi (mrd NOK)"]

#print(filtered[["År", "Eksportverdi (mrd NOK)"]])

# Tren modellen
model = LinearRegression()
model.fit(X, y)

# Lag fremtidige år (2026–2035)
framtid = pd.DataFrame({"År": list(range(2026, 2036))})
framtid["Predikert eksportverdi"] = model.predict(framtid)

fig.add_trace(go.Scatter(
    x=framtid["År"],
    y=framtid["Predikert eksportverdi"],
    name="Predikert eksportverdi (mrd NOK)",
    mode="lines+markers",
    line=dict(dash="dot", color="orange"),
    marker=dict(symbol="circle", size=6)
))


fig.update_layout(title="Fersk laks – Eksportverdi og prediksjon (2026–2035)")
fig.show()






#her brukes en csv fil og ikke api nøkkel for eksportverdidata siden dataen jeg afnt ikke hadde noen api nøkkel. 
#dette gjør at dataene ikke blir oppdatert i sanntid og derfor kan dataen utdateres. 
#api gjør at dataene blir oppdatert i sanntid og derfor er det bedre å bruke api enn csv fil.


In [22]:
import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# --- 1. Les eksportverdi fra CSV ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)
csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000

# --- 2. Hent fersk laks-data fra SSB API ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)

if response.status_code != 200:
    raise Exception(f"SSB API-feil: {response.status_code}")

data = response.json()
dimension_names = list(data["dimension"].keys())
values = data["value"]
dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
all_combinations = list(product(*dimensions))

df = pd.DataFrame(all_combinations, columns=dimension_names)
df["Value"] = values

df_pivot = df.pivot_table(
    index="Tid",
    columns=["VareGrupper2", "ContentsCode"],
    values="Value",
    aggfunc="sum"
).reset_index()

df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                    "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)
df_agg = df_agg[df_agg["År"] >= 2015]

# --- 3. Flett sammen eksportverdi og laksedata ---
merged = pd.merge(df_agg, csv_data[["År", "Eksportverdi (mill NOK)"]], on="År", how="left")
filtered = merged.dropna(subset=["Eksportverdi (mill NOK)", "Fersk laks - Kilospris"])

# --- 4. Regresjonsmodell: Kilospris → Eksportverdi ---
X = filtered[["Fersk laks - Kilospris"]]
y = filtered["Eksportverdi (mill NOK)"]
model = LinearRegression()
model.fit(X, y)

# Lag framtidige kilospriser (projiserte verdier) fra 2026 til 2035
# For enkelhet bruker vi gjennomsnittlig pris + små endringer
framtid_kilospris = np.linspace(X["Fersk laks - Kilospris"].min(), X["Fersk laks - Kilospris"].max() + 5, 10)
framtid = pd.DataFrame({"Fersk laks - Kilospris": framtid_kilospris})
framtid["Predikert eksportverdi"] = model.predict(framtid)




# --- Kombinert interaktiv graf ---
fig = go.Figure()

# Kilospris over tid
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Fersk laks - Kilospris"],
    name="Kilospris (kr/kg)",
    yaxis="y1",
    mode="lines+markers"
))

# Eksportverdi over tid
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Eksportverdi (mill NOK)"],
    name="Eksportverdi (mill NOK)",
    yaxis="y2",
    mode="lines+markers"
))

# Predikert eksportverdi basert på kilospris
fig.add_trace(go.Scatter(
    x=framtid["Fersk laks - Kilospris"],
    y=framtid["Predikert eksportverdi"],
    name="Predikert eksportverdi",
    xaxis="x2",  # Bruk sekundær x-akse
    yaxis="y2",
    mode="lines+markers",
    line=dict(dash="dot", color="orange")
))

# Layout med to x-akser og to y-akser
fig.update_layout(
    title="Fersk laks – Historisk og predikert eksportverdi",
    xaxis=dict(
        title="År",
        domain=[0, 0.7]
    ),
    xaxis2=dict(
        title="Kilospris (kr/kg)",
        domain=[0.75, 1],
        anchor="y2"
    ),
    yaxis=dict(
        title="Kilospris (kr/kg)",
        side="left"
    ),
    yaxis2=dict(
        title="Eksportverdi (mill NOK)",
        side="right",
        overlaying="y"
    ),
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=650
)

fig.show()


In [23]:
import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# --- 1. Les eksportverdi fra CSV ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)
csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000

# --- 2. Hent fersk laks-data fra SSB API ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)
data = response.json()

# Pakk ut data
dimension_names = list(data["dimension"].keys())
values = data["value"]
dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
all_combinations = list(product(*dimensions))

df = pd.DataFrame(all_combinations, columns=dimension_names)
df["Value"] = values

# Pivot tabellen
df_pivot = df.pivot_table(
    index="Tid",
    columns=["VareGrupper2", "ContentsCode"],
    values="Value",
    aggfunc="sum"
).reset_index()

df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                    "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)
df_agg = df_agg[df_agg["År"] >= 2015]

# --- 3. Flett dataene ---
merged = pd.merge(df_agg, csv_data[["År", "Eksportverdi (mill NOK)"]], on="År", how="left")
filtered = merged.dropna(subset=["Eksportverdi (mill NOK)", "Fersk laks - Kilospris"])

# --- 4. Tren regresjonsmodell ---
X = filtered[["Fersk laks - Kilospris"]]
y = filtered["Eksportverdi (mill NOK)"]

model = LinearRegression()
model.fit(X, y)

# --- 5. Prediker eksportverdi for framtidige år ---
framtid_år = list(range(filtered["År"].max() + 1, filtered["År"].max() + 11))
siste_pris = filtered["Fersk laks - Kilospris"].iloc[-1]
prisvekst = 0.5

framtid = pd.DataFrame({
    "År": framtid_år,
    "Fersk laks - Kilospris": [siste_pris + prisvekst * i for i in range(1, 11)]
})

framtid["Predikert eksportverdi"] = model.predict(framtid[["Fersk laks - Kilospris"]])

# --- 6. Plot alt i én graf ---
fig = go.Figure()

# Historisk eksportverdi
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Eksportverdi (mill NOK)"],
    name="Historisk eksportverdi",
    mode="lines+markers",
    line=dict(color="blue")
))

# Predikert eksportverdi
fig.add_trace(go.Scatter(
    x=framtid["År"],
    y=framtid["Predikert eksportverdi"],
    name="Predikert eksportverdi",
    mode="lines+markers",
    line=dict(color="orange", dash="dot")
))

fig.update_layout(
    title="Eksportverdi fra fersk laks – historisk og prediktiv analyse",
    xaxis_title="År",
    yaxis_title="Eksportverdi (mill NOK)",
    template="plotly_white",
    legend=dict(x=0.01, y=0.99),
    height=600
)

fig.show()


In [24]:
import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import numpy as np

# --- 1. Les eksportverdi fra CSV ---
csv_data = pd.read_csv("eksportverdi-laks-hittil-i-år-siste-10-år.csv")
csv_data.columns = ["År", "Eksportverdi (mrd NOK)"]
csv_data["År"] = csv_data["År"].astype(int)
csv_data["Eksportverdi (mill NOK)"] = csv_data["Eksportverdi (mrd NOK)"] * 1000

# --- 2. Hent fersk laks-data fra SSB API ---
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {"query": [], "response": {"format": "json-stat2"}}
response = requests.post(api_url, json=query)
data = response.json()

# Pakk ut data
dimension_names = list(data["dimension"].keys())
values = data["value"]
dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
all_combinations = list(product(*dimensions))

df = pd.DataFrame(all_combinations, columns=dimension_names)
df["Value"] = values

# Pivot tabellen
df_pivot = df.pivot_table(
    index="Tid",
    columns=["VareGrupper2", "ContentsCode"],
    values="Value",
    aggfunc="sum"
).reset_index()

df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                    "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)
df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)
df_agg = df_agg[df_agg["År"] >= 2015]

# --- 3. Flett dataene ---
merged = pd.merge(df_agg, csv_data[["År", "Eksportverdi (mill NOK)"]], on="År", how="left")
filtered = merged.dropna(subset=["Eksportverdi (mill NOK)", "Fersk laks - Kilospris"])

# --- 4. Tren regresjonsmodell ---
X = filtered[["Fersk laks - Kilospris"]]
y = filtered["Eksportverdi (mill NOK)"]

model = LinearRegression()
model.fit(X, y)

# --- 5. Lag prediksjon for fremtidige år ---
framtid_år = list(range(filtered["År"].max() + 1, filtered["År"].max() + 11))
siste_pris = filtered["Fersk laks - Kilospris"].iloc[-1]
prisvekst = 0.5  # kr/kg økning per år

framtid = pd.DataFrame({
    "År": framtid_år,
    "Fersk laks - Kilospris": [siste_pris + prisvekst * i for i in range(1, 11)]
})
framtid["Predikert eksportverdi"] = model.predict(framtid[["Fersk laks - Kilospris"]])

# --- 6. Kombinert graf med to y-akser ---
fig = go.Figure()

# Historisk kilospris
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Fersk laks - Kilospris"],
    name="Kilospris (kr/kg)",
    yaxis="y1",
    mode="lines+markers"
))

# Historisk eksportverdi
fig.add_trace(go.Scatter(
    x=filtered["År"],
    y=filtered["Eksportverdi (mill NOK)"],
    name="Eksportverdi (mill NOK)",
    yaxis="y2",
    mode="lines+markers"
))

# Predikert eksportverdi
fig.add_trace(go.Scatter(
    x=framtid["År"],
    y=framtid["Predikert eksportverdi"],
    name="Predikert eksportverdi",
    yaxis="y2",
    mode="lines+markers",
    line=dict(dash="dot", color="orange")
))

# Oppsett
fig.update_layout(
    title="Fersk laks – Kilospris og eksportverdi med prediktiv analyse",
    xaxis=dict(title="År"),
    yaxis=dict(title="Kilospris (kr/kg)", side="left"),
    yaxis2=dict(title="Eksportverdi (mill NOK)", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=650
)

fig.show()
